# 1. Introduzione
Come passo preliminare è bene riportare la definizione di violenza poliziesca, anche definita dal termine inglese *police brutality*, per comprendere l'oggetto dello studio. Si tratta di un insieme di comportamenti caratterizzati da violenza, repressione, abuso di potere, corruzione, abusi sessuali, eccessivo uso della forza, discriminazioni razziste e intimidazioni di cui responsabili uno o più individui operanti nelle forze di polizia. Lo studio nasce, dunque, con lo scopo di individuare l'eventuale esistenza di un pregiudizio socio-culturale da parte della polizia americana nei confronti delle minoranze etniche in particolar modo afroamericani, ispanici, asiatici e nativi (nativi americani e nativi delle isole del Pacifico). <br>
Per affrontare questo argomento ci si è focalizzati principalmente sui dati raccolti dalla testata giornalistica *The Washington Post* [1], e dalla ricerca *Mapping Police Violence* [2].

****

## 1.1 Pulizia sui dataset

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
police_shootings_data_wp = pd.read_csv('../input/fatal-police-shootings-data/fatal-police-shootings-data.csv')

police_shootings_data_mpv = pd.read_csv('../input/police-shootings-mpv/police-shootings-mpv.csv')

Nella fase di pulizia bisognerà focalizzarsi su ciò che è realmente utile ai fini dello studio condotto. Le informazioni rilevanti verranno tutte conservate all'interno di un unico dataset, risultato della fusione di entrambi i dataset elaborati. <br>
La prima fase di pulizia riguarderà *police_shootings_data_wp*; verranno eliminate le colonne `county`, `location_precision`, `race_source`, `was_mental_illness_related`, `body_camera`, `agency_ids`.

In [ ]:
# 1.

police_shootings_data_wp_v1 = police_shootings_data_wp[police_shootings_data_wp.columns[~police_shootings_data_wp.columns.isin(['county', 'location_precision', 'race_source', 
                                                                                                       'was_mental_illness_related', 'body_camera', 'agency_ids'])]]
police_shootings_data_wp_v1.head()

Lo stesso viene effettuato per *police_shootings_data_mpv*. Vengono mantenute solamente le colonne più importanti: `name`, `age`, `gender`, `race`, `date`, `city`, `state`, `cause_of_death`, `allegedly_armed`, `wapo_armed`, `wapo_threat_level`, `wapo_flee`, `latitude` e `longitude`.

In [ ]:
# 1.

police_shootings_data_mpv_v1 = police_shootings_data_mpv[police_shootings_data_mpv.columns[police_shootings_data_mpv.columns.isin(['name', 'age', 'gender', 'race', 'date', 'city', 'state', 'cause_of_death', 
                                                                                                       'allegedly_armed', 'wapo_armed', 'wapo_threat_level', 'wapo_flee', 'latitude', 'longitude'])]]


police_shootings_data_mpv_v1.head()

All'interno di entrambi i dataset in corrispondenza della colonna `race`, l'interessata principale dello studio, bisogna rimuovere i record con valore `--` dal primo dataset (*Washington Post*) e `Unknown race` dal secondo dataset (*MVP*) poiché si vogliono studiare solo i casi in cui l'etnia di provenienza è conosciuta.

In [ ]:
# 2. 

print("Records con Unknown Race: " + str(len(police_shootings_data_wp_v1)))

police_shootings_data_wp_v2 = police_shootings_data_wp_v1.drop(police_shootings_data_wp_v1[police_shootings_data_wp_v1.race.isna()].index)

police_shootings_data_wp_v2 = police_shootings_data_wp_v2.reset_index(drop=True)

print("Records senza Unknown Race: " + str(len(police_shootings_data_wp_v2)))
police_shootings_data_wp_v2.head()


In [ ]:
# 2 

print("Records con Unknown Race: " + str(len(police_shootings_data_mpv_v1)))

police_shootings_data_mpv_v2 = police_shootings_data_mpv_v1.drop(police_shootings_data_mpv_v1[police_shootings_data_mpv_v1.race == "Unknown race"].index)
police_shootings_data_mpv_v2 = police_shootings_data_mpv_v1.drop(police_shootings_data_mpv_v1[police_shootings_data_mpv_v1.race.isna()].index)

police_shootings_data_mpv_v2 = police_shootings_data_mpv_v2.reset_index(drop=True)

print("Records senza Unknown Race: " + str(len(police_shootings_data_mpv_v2)))
police_shootings_data_mpv_v2.head()

Si effettua una seconda operazione di format sulla colonna `date` appartenente al secondo dataset. L'obiettivo è di utilizzare il format **YYYY-MM-DD** che rispetta il formato del primo dataset.

In [ ]:
import datetime

police_shootings_data_mpv_v2['date'] = pd.to_datetime(police_shootings_data_mpv_v2['date'], dayfirst = True).dt.strftime('%Y-%m-%d')

police_shootings_data_mpv_v2.head()

****

## 1.2 Dataset Merge  

In questa fase viene effettuato il merge sui due dataset `police_shootings_data_mpv_v2` e `police_shootings_data_wp_v2` mantenendo le colonne `name`, `state`, `age` e `date`, le quali presenziano in entrambi i dataset. Questa operazione permette di contemplare un singolo dataset contenente tutti i record registrati dal 2013 fino al 2024. Inoltre rende i casi di individui con nomi omonimi univoci, poiché caratterizzati da data, stato in cui si è registrata la morte, età ed etnia. Il risultato sarà dunque il seguente:

In [ ]:
# 2 -> # 2.1 -> # 3                  
police_shootings = pd.merge(police_shootings_data_mpv_v2, police_shootings_data_wp_v2, how = "outer", on = ['name', 'state', 'age', 'date'])

print("Record totali: " + str(len(police_shootings)))
police_shootings.head()

Un esempio di nome omonimo che appare all'interno del dataset è il record con valore `Robert Edwards` che appare tre volte (due volte nel dataset del *Washington Post* e una volta in *MPV*). Qui di seguito viene mostrato come viene individuato univocamente, dopo il merge, all'interno di `police_shootings`.

In [ ]:
police_shootings[police_shootings['name'] == "Robert Edwards"].head() 

****

## 1.3 Pulizia su Police Shootings 

Dai precedenti risultati è possibile notare che le colonne `gender_x`, `race_x`, `city_x`, `city_y`, `gender_y` e `race_y`, si riferiscono alle stesse informazioni. Nello stesso caso ricadono anche le colonne `wapo_armed` e `armed_with`, `wapo_flee` e `flee_status`, `wapo_threat_level` e `threat_type`. Si effettuano dunque delle operazioni che immettono tutte le informazioni in un'unica colonna.

In [ ]:
# 3.1

police_shootings['city'] = police_shootings['city_x'].fillna(police_shootings['city_y'])
police_shootings['race'] = police_shootings['race_x'].fillna(police_shootings['race_y'])
police_shootings['gender'] = police_shootings['gender_x'].fillna(police_shootings['gender_y'])
police_shootings['flee_status'] = police_shootings['wapo_flee'].fillna(police_shootings['flee_status'])
police_shootings['threat_type'] = police_shootings['wapo_threat_level'].fillna(police_shootings['threat_type'])
police_shootings['armed_with'] = police_shootings['wapo_armed'].fillna(police_shootings['armed_with'])
police_shootings['latitude'] = police_shootings['latitude_x'].fillna(police_shootings['latitude_y'])
police_shootings['longitude'] = police_shootings['longitude_x'].fillna(police_shootings['longitude_y'])

Si eliminano dunque le colonne ridondanti in modo da conservare solamente le colonne appena create.

In [ ]:
police_shootings = police_shootings[police_shootings.columns[~police_shootings.columns.isin(['gender_x', 'gender_y', 'race_x', 'race_y', 'date_x', 'date_y', 
                                                                                                'city_x', 'city_y', 'id', 'wapo_flee', 'latitude_x',
                                                                                                'latitude_y', 'longitude_x', 'longitude_y',
                                                                                                 'wapo_threat_level', 'wapo_armed'])]]

police_shootings.head()

In seguito alla fusione delle varie colonne contenenti le medesime informazioni, bisogna adesso modellare la formattazione di alcuni valori che riferiscono la medesima informazione. Un esempio è dato dal valore `car` individuato all'interno della colonna `flee_status`: questo valore è appartenente al primo dataset, mentre `Car` appartiene al secondo dataset. In questo caso è necessario modificare solamente la prima lettera. <br>
In altri casi  bisognerà trasformare il valore del primo dataset in quello del secondo dataset, come ad esempio all'interno della colonna `threat_type` in cui il valore `move` e `Sudden Threatening Movement` indicano la stessa azione. <br>
Effettuate queste operazioni verrà mostrata la lista dei valori presenti nelle corrispettive colonne.

In [ ]:
# 3.2
police_shootings['flee_status'] = police_shootings['flee_status'].replace({'car': 'Car', 'other': 'Other', 'not': 'Not Fleeing',
                                                                              'foot': 'Foot', 'Car, Foot': 'Car',
                                                                                'Foot, Car': 'Foot', 'Motorcycle, Foot': 'Motorcycle',
                                                                                  'Other': np.NaN})

police_shootings['threat_type'] = police_shootings['threat_type'].replace({'shoot': 'Used Weapon', 'point': 'Brandished Weapon', 
                                                                                'attack': 'Attack', 'Unclear': np.NaN,
                                                                                  'move': 'Sudden Threatening Movement',
                                                                                      'Other': np.NaN, 'undetermined': np.NaN,
                                                                                    'Undetermined': np.NaN})

In [ ]:
print(set(police_shootings.get('flee_status').tolist()))

In [ ]:
print(set(police_shootings.get('threat_type').tolist()))

Si continua il lavoro di pulizia sulla colonna contenente la caratteristica principale dello studio, la colonna `race`. Il valore veniva indicato nel dataset `police_shootings_mpv` per esteso (*es. White, Black etc.*). Viene trasformato nel format del secondo dataset abbreviando alla prima lettera. In questo modo il valore `W` starà per *White*, `B` per *Black*, `A` per *Asian*, `H` per *Hispanic* e `N` per *Native American & Native Hawaiian and Pacific Islander*.

In [ ]:
police_shootings['race'] = police_shootings['race'].astype(str).str[0]

police_shootings.head()

In [ ]:
print(set(police_shootings.get('race').tolist()))

Si noti che dai valori mostrati sono ancora presenti i due valori `U` e `O`, rispettivamente *Unknown* e *Other*. Quest'ultimi non sono utili ai fini dello studio pertanto verranno eliminati i record contenenti tali valori.

In [ ]:
police_shootings = police_shootings.drop(police_shootings[police_shootings.race == "U"].index)
police_shootings = police_shootings.drop(police_shootings[police_shootings.race == "O"].index)

police_shootings = police_shootings.reset_index(drop=True)

In [ ]:
print(set(police_shootings.get('race').tolist()))

Un'ultima operazione di pulizia deve essere effettuata su `gender`. Trattandosi di un valore di tipo *String* sarà solamente necessario rendere il primo carattere minuscolo. <br>
Si visualizza un'ultima volta il numero di record presenti nel dataset dopo questa operazione di *drop*.

In [ ]:
police_shootings['gender'] = police_shootings['gender'].str.lower()

police_shootings.head()

In [ ]:
print("Record totali: " + str(len(police_shootings)))
police_shootings.head()

****

# 2. Analisi

In questa fase si descriveranno alcune soluzioni di analisi statistica con l'obiettivo di evidenziare risultati che possano dare delle concrete fondamenta alla tesi di partenza esposta nell'introduzione allo studio. Ogni operazione e risultato visualizzato sarà da considerarsi sul nuovo dataset generato e adesso completo di pulizia `police_shootings`. <br>

## 2.1 Etnie 

Qui di seguito il primo plot ad essere elaborato visualizza la distribuzione delle etnie nel dataset. Questo permette di comprendere l'etnia più diffusa all'interno dei record: il valore `W` con 7159 record. Le minoranze etniche rappresentate da `B`, `H`, `N`, e `A` sono un numero pari a 7375 record. Considerando che sono stati eliminati record di cui non era possibile conoscere l'etnia oppure in cui il valore veniva indicato come *Other*, la differenza tra le minoranze etniche raggruppate e i bianchi non è sostanziale, dunque lo studio sarà bilanciato. 

In [ ]:
from matplotlib import pyplot as plt

police_shootings['race'].value_counts()

In [ ]:
police_shootings['race'].value_counts().plot.bar()

## 2.2 Stati

Il secondo plot ad essere mostrato è una *Choropleth map* [3]. Quest'ultima consiste in un tipo di mappa statistica che utilizza un'intensità diversa di colore per area geografica, a seconda della scala di colori utilizzata. L'intensità dipende da una somma totale delle caratteristiche scelte per una certa area come può essere ad esempio la densità della popolazione situata in un certo luogo. Nel caso dello studio qui riportato l'elemento caratterizzante è il numero di uccisioni in un determinato stato degli US.

In [ ]:
import plotly.graph_objs as go 
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [ ]:
df = police_shootings['state'].value_counts().reset_index().rename(columns={"index": "state", 0: "count"})

In [ ]:
data = dict(
        type = 'choropleth',
        colorscale = 'Viridis',
        reversescale = True,
        locations = df['state'],
        locationmode = "USA-states",
        z = df['count'].astype(float),
        marker = dict(line = dict(color = 'rgb(255, 255, 255)', width = 1)),
        colorbar = {'title' : 'No. kills registered across the States 2013-2024'},
      ) 

In [ ]:
layout = dict(geo = dict(scope='usa',
                showlakes = True,
                lakecolor = 'rgb(85,173,240)')
             )

In [ ]:
choromap = go.Figure(data = [data],layout = layout)
iplot(choromap, validate=False)

Dalla grafica è possibile vedere che gli stati con il maggior numero di uccisioni risultano essere `CA`: California, `TX`: Texas e `FL`: Florida. Le morti registrate coprono un intervallo di tempo a partire dal `01-01-2013` al `10-05-2024`. <br> <br>
Verrà adesso considerata e calcolata un'analisi che permetterà di individuare le aree con un'alta concentrazione di data points. In questo caso i data points vengono caratterizzati da una precisa latitudine e longitudine. Si tratta della tecnica di *Density Estimation*; permetterà dunque di visualizzare le aree con una maggiore numerosità con un colore più caldo, viceversa le aree con minore densità verranno rappresentate con un colore più tenue. Bisognerà anzitutto assicurarsi che il dataset non abbia al suo interno valori `NA`, in particolar modo per quel che riguarda `latitude` e `longitude`.

In [ ]:
from scipy.stats import gaussian_kde
import geopandas as gpd
from shapely.geometry import Polygon

police_shootings = police_shootings.replace([np.inf, -np.inf], np.nan).dropna()

Viene utilizzato un metodo non parametrico della *Density Estimation*, in modo da concentrarsi unicamente sui dati già esistenti, senza fare alcuna assunzione sulla distribuzione dei dati. Viene, dunque, scelta una *Kernel Density Estimation*, **KDE**, con kernel gaussiano, basandosi sul metodo di scott per la scelta della bandwith. La scelta della bandwith rende la stima più o meno sensibile alle variazioni dei dati e dunque una buona scelta renderà la stima più stabile. Nel seguente codice si modifica il plot in modo da mostrare solamente gli stati americani.

In [ ]:
latitudes = police_shootings['latitude'].values
longitudes = police_shootings['longitude'].values

# KDE
xy = np.vstack([longitudes, latitudes])
kde = gaussian_kde(xy, bw_method = 'scott')

cities = {
    'New York City': (-74.0060, 40.7128),
    'Los Angeles': (-118.2437, 34.0522),
    'Chicago': (-87.6298, 41.8781),
    'Houston': (-95.3698, 29.7604),
    'Phoenix': (-112.0740, 33.4484),
    'San Antonio': (-98.4936, 29.4241),
    'San Diego': (-117.1611, 32.7157),
    'Dallas': (-96.7970, 32.7767),
    'Seattle': (-122.3321, 47.6062),
    'Portland': (-122.6765, 45.5234),
    'Denver': (-104.9903, 39.7392),
    'Boston': (-71.0589, 42.3601),
    'Washington D.C.': (-77.0369, 38.9072),
    'Minneapolis': (-93.2650, 44.9778),
    'Detroit': (-83.0458, 42.3314),
    'Miami': (-80.1918, 25.7617),
    'Atlanta': (-84.3880, 33.7490),
    'Charlotte': (-80.8431, 35.2271),
    'Raleigh': (-78.6382, 35.7796),
    'Nashville': (-86.7816, 36.1627),
    'Orlando': (-81.3792, 28.5383)
}

# Griglia per l'area di interesse (United States)
lon_min, lon_max = -130, -65  # Limiti approssimati per gli Stati Uniti
lat_min, lat_max = 25, 50

lon_grid, lat_grid = np.meshgrid(np.linspace(lon_min, lon_max, 100), np.linspace(lat_min, lat_max, 100))
grid_coords = np.vstack([lon_grid.ravel(), lat_grid.ravel()])

# KDE sulla griglia
density = kde(grid_coords).reshape(lon_grid.shape)

# Shapefile di Geopandas per i vari paesi
shapefile_path = '/kaggle/input/map-test/ne_110m_admin_0_countries.shp'
world = gpd.read_file(shapefile_path)

# Filtro per selezionare solamente gli Stati Uniti
usa = world[world['ADMIN'] == 'United States of America'].to_crs('EPSG:4326')
bbox = Polygon([(-125, 25), (-65, 25), (-65, 50), (-125, 50)])

usa_clipped = gpd.clip(usa, bbox)

fig, ax = plt.subplots(figsize = (10, 8))

# Mostra il plot che contiente solo gli US
usa_clipped.plot(ax = ax, color = 'whitesmoke', edgecolor = 'black')

# Plot per la riga verticale della KDE 
contour = ax.contourf(lon_grid, lat_grid, density, cmap = 'Reds', levels = 15, alpha = 0.6)
cbar = plt.colorbar(contour, ax = ax, orientation = 'vertical', fraction = 0.046, pad = 0.04)
cbar.set_label('Density')

# Plot per le città maggiori
for city, (lon, lat) in cities.items():
    ax.plot(lon, lat, 'ro')  # red dot for city location
    ax.text(lon, lat, city, fontsize = 10, ha = 'right')

plt.title('Kernel Density Estimation (KDE) of Latitude and Longitude in United States')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

Si nota una maggiore concentrazione di densità nello Stato della `California`, nello specifico nelle città principali come `Los Angeles`, `San Diego` e `Phoenix`. Un'altra area con discreta concentrazione di densità è l'insieme degli Stati formati da `Georgia`, `Tennessee` e `Carolina del Nord`. Le principali città rispettivamente per ciascuna sono: `Nashville`, `Atlanta`, e `Charlotte` con `Raleigh`. Un'area simile è individuata anche nei pressi dello Stato di `New York` con `New York City` e `Maryland` con il `District of Columbia`, la sede di `Washington D.C`. L'ultima area con una concentrazione di densità meno rimarcata rispetto le precedenti è lo Stato del `Texas`, con `Houston`. Questi risultati potrebbero in qualche modo suggerire un'inclinazione più violenta nella zona Sud Ovest degli Stati Uniti che si ritrova anche nella zona Nord Est. Si noti invece i risultati che si ottengono a Nord/Nord Ovest. Quest'area sembra molto meno soggetta a violenze e crimini contrariamente a quanto visto prima. Considerazioni sulla latitudine e la posizione geografica verranno affrontate nel paragrafo **2.6** dell'analisi. <br>
Si procede adesso con lo studio sulle cause di morte laddove queste siano riportate tramite valori non nulli, per i vari individui presenti all'interno del dataset.

## 2.3 Cause della morte

Il terzo plot mostrato è una *Pie chart*. Si tratta di un grafico a torta che permette di visualizzare le maggiori cause che hanno portato alla morte delle vittime, registrate dal 2013 al 2024. 

In [ ]:
df_cause = police_shootings['cause_of_death'].value_counts().reset_index().rename(columns = {"index": "armed", 0: "count"})

df_cause

In [ ]:
threshold = 0.01 # utile per sommare le rimanenti categorie e visualizzare solo le maggiori

df_cause['percentage'] = df_cause['count'] / df_cause['count'].sum()
remaining = df_cause.loc[df_cause['percentage'] < threshold].sum(axis = 0)
remaining.loc['cause_of_death'] = 'Others'
df_cause = df_cause[df_cause['percentage'] >= threshold]

df_cause = pd.concat([df_cause, pd.DataFrame([remaining])], ignore_index = True)

df_cause

In [ ]:
plt.figure(figsize = (5,5))

colors = sns.color_palette("Set2")
plt.pie(df_cause['count'],
        autopct = '%.1f%%',
        explode = [0.05, 0.05, 0.05],
        labels = df_cause['cause_of_death'], colors = colors)


plt.title(
    label = "Cause of death in the US 2013-2024", 
    fontdict = {"fontsize":12},
)

plt.show()

Nella stragrande maggioranza dei casi (**96,2 %**), la causa della morte di un individuo è un'arma da fuoco, `Gunshot`. Si vedono anche altri casi minori in cui la morte è provocata da una combinazione di un'arma da fuoco e l'uso del `Taser` (**2,5 %**). In ultima battuta vengono raggruppati i casi minori, `Others` (**1,3 %**), in cui il risultato è una combinazione delle precedenti o un attacco fisico alla persona. <br>
Questo potrebbe suggerire una possibile correlazione tra la libera detenzione di armi negli US e l'aumento di casi di violenza da parte della polizia. Trattandosi di un fattore socio-culturale che ha radici profonde potrebbe spiegare il motivo per il quale il numero di uccisioni è così alto ed è quasi sempre dato da un'arma da fuoco. Tuttavia, bisogna considerare il reale contesto in cui gli operatori di polizia si ritrovano e la relativa minaccia che spiegherebbe l'inevitabile uso di una pistola.

## 2.4 Threat type e Flee status

Si prendono adesso in considerazione i tre stati con più morti registrate, dunque, in ordine: la California, il Texas e la Florida. Essendo gli Stati Uniti d'America una repubblica di tipo federale, ogni Stato membro conserva parte della propria sovranità, pertanto in ciascuno degli Stati vige un sistema differente. Questo potrebbe rispondere al motivo per cui si registrano più morti in uno Stato rispetto ad un altro. Tra i diversi motivi bisogna considerare anche la posizione geografica, ad esempio i confini con altri paesi e, come menzionato nel paragrafo precedente, il relativo rischio all'interno di un contesto e le condizioni di fuga di un soggetto. <br>
Per analizzare, dunque, il rischio relativo si prende in considerazione la colonna `threat_type` nel quale compaiono i valori `Attack`, `Brandished Weapon`, `Sudden Threating Movement`, `Used Weapon`, `accident`, `flee` ed infine `threat`. Tra tutti i valori elencati, `Attack` è il livello di pericolo più elevato. Per quanto riguarda le condizioni di fuga sarà considerata la colonna `flee_status` dove i valori `Car`, `Foot`, `Motorcycle`, `Other`, `bicycle`, e `tractor` verranno trasformati in condizioni di fuga: `Fleeing`. In questo modo la colonna `flee_status` sarà caratterizzata solamente dai valori `Not Fleeing` e `Fleeing`. <br>
La medesima condizione di normalizzazione avverrà sulla colonna `allegedly_armed` in cui i valori `Vehicle` e `Unclear` verranno trasformati rispettivamente in `Allegedly Armed` e `Unarmed/Did Not Have Actual Weapon`. <br>
*N.B. Le approssimazioni delle percentuali sono state effettuate, laddove possibile, in eccesso. Le normalizzazioni all'interno della funzione* `crosstab` *vengono invece effettuate sulle righe in modo da mettere in risalto nella prima visualizzazione lo stato di Attack e nella seconda visualizzazione lo stato di Allegedly Armed e Unarmed/Did Not Have Actual Weapon.*

In [ ]:
police_shootings['flee_status'] = police_shootings['flee_status'].replace({'Car': 'Fleeing', 'Foot': 'Fleeing',
                                        'Motorcycle': 'Fleeing', 'Other': 'Fleeing',
                                        'bicycle': 'Fleeing', 'tractor': 'Fleeing'})

In [ ]:
police_shootings['allegedly_armed'] = police_shootings['allegedly_armed'].replace({'Vehicle': 'Allegedly Armed', 
                                                                             'Unclear': 'Unarmed/Did Not Have Actual Weapon'})

### 2.4.1 California

Nello stato della California le situazioni di reale pericolo descritte dal valore `Attack` sono, in percentuale sull'etnia, le seguenti:
- Lo **0,3 %** delle situazioni di Attack è causato da `A`. 
- Il **19 %** delle situazioni di Attack è causato da `B`.
- Il **42 %** delle situazioni di Attack è causato da `H`.
- Lo **0,2 %** delle situazioni di Attack è causato da `N`.
- Il **32 %** delle situazioni di Attack è causato da `W`.

Sembrerebbe dunque che le maggiori situazioni di pericolo si verifichino per mano degli ispanici. La percentuale quasi vicina alla metà dei casi della popolazione ispanica registrata nello stato della California, renderebbe la risposta dei poliziotti legittima, in quanto coinvolti in un contesto pericoloso per la loro vita e la sicurezza dei cittadini. Tuttavia, ancora una volta, bisogna considerare la percentuale di persone non armate con le relative condizioni di fuga prima di trarre conclusioni.

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "CA"]['threat_type'], police_shootings['race'], normalize = 'index')

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "CA"]['threat_type'], police_shootings['race'], normalize = 'index').T.plot.bar(stacked = True)

Di seguito è possibile notare che il **18%** dei casi registrati non armati si trova nello stato di fuga, mentre l' **82%** dei casi registrati non armati non si trova nelle condizioni di fuga. Questa percentuale non dovrebbe esistere poiché se un soggetto non è armato e tanto più non fugge, non si presenta alcuna situazione di pericolo considerevole, e non ha motivo di essere attaccato.

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "CA"]['allegedly_armed'], police_shootings['flee_status'], normalize = 'index')

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "CA"]['allegedly_armed'], police_shootings['flee_status'], normalize = 'index').T.plot.bar(stacked = True)

### 2.4.2 Texas

Nello stato del Texas le situazioni di reale pericolo descritte dal valore `Attack` sono, in percentuale sull'etnia, le seguenti:
- Il **3 %** delle situazioni di Attack è causato da `A`. 
- Il **25 %** delle situazioni di Attack è causato da `B`.
- Il **32 %** delle situazioni di Attack è causato da `H`.
- Lo **0,2 %** delle situazioni di Attack è causato da `N`.
- Il **39 %** delle situazioni di Attack è causato da `W`.

Variando la tipologia di popolazione la situazione è diversa rispetto al primo Stato analizzato. Le situazioni di pericolo sono esistenti, ma si analizza successivamente la percentuale di persone non armate e il loro stato di fuga.

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "TX"]['threat_type'], police_shootings['race'], normalize = 'index')

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "TX"]['threat_type'], police_shootings['race'], normalize = 'index').T.plot.bar(stacked = True)

Di seguito è possibile notare che il **27%** dei casi registrati non armati è anche nello stato di fuga, mentre il **73%** dei casi registrati non armati non si trova nelle condizioni di fuga. 

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "TX"]['allegedly_armed'], police_shootings['flee_status'], normalize = 'index')

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "TX"]['allegedly_armed'], police_shootings['flee_status'], normalize = 'index').T.plot.bar(stacked = True)

### 2.4.3 Florida

Per ultimo, nello stato della Florida le situazioni di reale pericolo descritte dal valore `Attack` sono, in percentuale sull'etnia, le seguenti:
- L' **1 %** delle situazioni di Attack è causato da `A`. 
- Il **33 %** delle situazioni di Attack è causato da `B`.
- Il **13 %** delle situazioni di Attack è causato da `H`.
- Il **52 %** delle situazioni di Attack è causato da `W`.

Anche in questo caso, variando la tipologia di popolazione la situazione è diversa rispetto agli Stati precedenti. Da notare inoltre l'assenza della popolazione Nativo Americana e delle Isole Pacifiche per via di valori non registrati sulla colonna `threat_type`. Le situazioni di pericolo sono esistenti in maggior numero dalla popolazione `W`, ma si analizza successivamente la percentuale di persone non armate e il loro stato di fuga.

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "FL"]['threat_type'], police_shootings['race'], normalize = 'index')

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "FL"]['threat_type'], police_shootings['race'], normalize = 'index').T.plot.bar(stacked = True)

Di seguito è possibile notare che il **34%** dei casi registrati non armati è anche nello stato di fuga, mentre il **66%** dei casi registrati non armati non si trova nelle condizioni di fuga. 

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "FL"]['allegedly_armed'], police_shootings['flee_status'], normalize = 'index')

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "FL"]['allegedly_armed'], police_shootings['flee_status'], normalize = 'index').T.plot.bar(stacked = True)

### 2.4.4 Confronto tra Unarmed/Race nei tre Stati

Si confronta adesso la condizione di detenzione arma messa in relazione alla specifica etnia di provenienza. Quest'ultima verrà espressa in percentuale. Si sceglie una normalizzazione sulle colonne `Allegedly Armed` e `Unarmed/Did Not Have Actual Weapon`.

In [ ]:
from scipy.stats import chi2_contingency

Nello stato della California:
- Il **2 %** delle persone non armate/senza una vera e propria arma è `A` contro il **4 %** di persone `A` dichiarate armate. 
- Il **22 %** delle persone non armate/senza una vera e propria arma è `B` contro il **17 %** di persone `B` dichiarate armate.
- Il **40 %** delle persone non armate/senza una vera e propria arma è `H` contro il **44 %** di persone `H` dichiarate armate.
- Il **2 %** delle persone non armate/senza una vera e propria arma è `N` contro il **1 %** di persone `N` dichiarate armate.
- Il **33 %** delle persone non armate/senza una vera e propria arma è `W` contro il **32 %** di persone `W` dichiarate armate.

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "CA"]['race'], police_shootings['allegedly_armed'], normalize = 1)

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "CA"]['race'], police_shootings['allegedly_armed'], normalize = 1).T.plot.bar(stacked=True)

In [ ]:
contingency_table = pd.crosstab(police_shootings[police_shootings['state'] == "CA"]['race'], police_shootings['allegedly_armed'])

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi2:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("Expected frequencies:", expected)

Nello stato del Texas:
- Lo **0 %** delle persone non armate/senza una vera e propria arma è `A` contro il **3 %** di persone `A` dichiarate armate. 
- Il **23 %** delle persone non armate/senza una vera e propria arma è `B` contro il **25 %** di persone `B` dichiarate armate.
- Il **27 %** delle persone non armate/senza una vera e propria arma è `H` contro il **34 %** di persone `H` dichiarate armate.
- Lo **0 %** delle persone non armate/senza una vera e propria arma è `N` contro lo **0,1 %** di persone `N` dichiarate armate.
- Il **49 %** delle persone non armate/senza una vera e propria arma è `W` contro il **36 %** di persone `W` dichiarate armate.

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "TX"]['race'], police_shootings['allegedly_armed'], normalize = 1)

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "TX"]['race'], police_shootings['allegedly_armed'], normalize = 1).T.plot.bar(stacked=True)

In [ ]:
contingency_table = pd.crosstab(police_shootings[police_shootings['state'] == "TX"]['race'], police_shootings['allegedly_armed'])

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi2:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("Expected frequencies:", expected)

Nello stato della Florida:
- Il **2 %** delle persone non armate/senza una vera e propria arma è `A` contro l'**1 %** di persone `A` dichiarate armate. 
- Il **28 %** delle persone non armate/senza una vera e propria arma è `B` contro il **30 %** di persone `B` dichiarate armate.
- Il **18 %** delle persone non armate/senza una vera e propria arma è `H` contro il **12 %** di persone `H` dichiarate armate.
- Il **18 %** delle persone non armate/senza una vera e propria arma è `N` contro il **12 %** di persone `N` dichiarate armate.
- Il **52 %** delle persone non armate/senza una vera e propria arma è `W` contro il **55 %** di persone `W` dichiarate armate.

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "FL"]['race'], police_shootings['allegedly_armed'], normalize = 1)

In [ ]:
pd.crosstab(police_shootings[police_shootings['state'] == "FL"]['race'], police_shootings['allegedly_armed'], normalize = 1).T.plot.bar(stacked=True)

In [ ]:
contingency_table = pd.crosstab(police_shootings[police_shootings['state'] == "FL"]['race'], police_shootings['allegedly_armed'])

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi2:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("Expected frequencies:", expected)

### 2.4.5 Test Chi-Quadrato di Pearson su Race/State

I risultati precedenti del test di indipendenza del Chi-quadrato per la verifica di associazioni tra variabili non hanno fornito risultati particolarmente significativi. Si ritenta l'applicazione del test, questa volta su Stato/Etnia, per verificare se esiste una relazione il numero di uccisioni registrate di una certa etnia e lo Stato in cui occorrono. 

In [ ]:
contingency_table = pd.crosstab(index=police_shootings['state'] == "CA", columns=police_shootings['race'] == "H")

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi2:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("Expected frequencies:", expected)

In [ ]:
contingency_table = pd.crosstab(index=police_shootings['state'] == "TX", columns=police_shootings['race'] == "H")

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi2:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("Expected frequencies:", expected)

In [ ]:
contingency_table = pd.crosstab(index=police_shootings['state'] == "FL", columns=police_shootings['race'] == "H")

chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi2:", chi2)
print("p-value:", p)
print("Degrees of freedom:", dof)
print("Expected frequencies:", expected)

Si può concludere che nei tre Stati con maggiori morti osservati, l'etnia di tipo ispanico `H` è significativamente correlata allo Stato *(p-value inferiore a 0,05)*, il che potrebbe suggerire una discriminazione a discapito di questa popolazione. Per approfondire ulteriormente questa conclusione si misura nel paragrafo successivo un rate di uccisioni per etnie nei tre Stati osservati.

## 2.5 Rate di uccisioni per etnie nei tre Stati

Si prendono in considerazione le tre etnie più diffuse negli Stati Uniti, i bianchi, gli ispanici e gli afro-americani; rispettivamente `W`, `H` e `B`. In particolar modo nei tre Stati con più morti registrate, California, Texas e Florida. <br>
L'obiettivo di quest'ultima visualizzazione statistica è incentrato sul mostrare la distribuzione delle morti su 100,000 persone per varia etnia. Per farlo verrà consultata la pagina *Census Reporter*[4]: un progetto free ed open-source che raccoglie le informazioni relative al censimento statunitense con dati aggiornati fino all'anno 2022. Per ciascun Stato, dunque, verrà raccolto il dato sul totale della popolazione relativa all'etnia di appartenenza; successivamente verrà calcolato un rate di uccisioni per etnia conteggiando il numero di morti per la specifica etnia messo a rapporto con il totale della popolazione per quell'etnia. Infine il risultato sarà moltiplicato per 100,000.

$$
  \frac{\text{No. kills of an ethnicity}}{\text{Tot. of ethnicity population}}*100,000
$$

*N.B: Il risultato verrà arrotondato per eccesso se supera la metà compresa dei decimali esclusi (si approssima alla prima cifra), e per difetto se è al di sotto di questa metà*

In [ ]:
state_1 = police_shootings[police_shootings['state'] == "CA"]

h_counts = state_1['race'].value_counts()['H']
w_counts = state_1['race'].value_counts()['W']
b_counts = state_1['race'].value_counts()['B']

#popestimate 2022
pop_black = 2025218 
pop_hispanic = 15732184
pop_white = 13160426

data = {
    'ethnicity': ['Hispanic', 'White', 'African American'],
    'homicides': [h_counts, w_counts, b_counts],
    'population': [pop_hispanic, pop_white, pop_black]
}
df_to_visualize = pd.DataFrame(data)

df_to_visualize['homicide_rate_per_100k'] = df_to_visualize['homicides'] / df_to_visualize['population'] * 100000

Questo risultato mostra che in California su **100,000 persone** vengono uccise:
- 2 persone ispaniche
- 2 persone bianche
- 7 persone afro-americane

In [ ]:
plt.figure(figsize = (5, 5))
plt.bar(df_to_visualize['ethnicity'], df_to_visualize['homicide_rate_per_100k'], color = ['blue', 'green', 'red'])
plt.xlabel('Ethnicity')
plt.ylabel('Homicide Rate per 100,000 People')
plt.title('Homicide Rate per 100,000 People by Ethnicity')
plt.ylim(0, max(df_to_visualize['homicide_rate_per_100k']) * 1.2)  # Aggiunge un po' di spazio sopra la barra più alta
plt.show()

In [ ]:
state_2 = police_shootings[police_shootings['state'] == "TX"]

h_counts = state_2['race'].value_counts()['H']
w_counts = state_2['race'].value_counts()['W']
b_counts = state_2['race'].value_counts()['B']

#popestimate 2022
pop_black = 3508706
pop_hispanic = 12070642
pop_white = 11683430

df_to_visualize['homicides'] = [h_counts, w_counts, b_counts]
df_to_visualize['population'] = [pop_hispanic, pop_white, pop_black]

df_to_visualize['homicide_rate_per_100k'] = df_to_visualize['homicides'] / df_to_visualize['population'] * 100000

Questo risultato mostra che in Texas su **100,000 persone** vengono uccise:
- 2 persone ispaniche
- 2 persone bianche
- 4 persone afro-americane

In [ ]:
plt.figure(figsize = (5, 5))
plt.bar(df_to_visualize['ethnicity'], df_to_visualize['homicide_rate_per_100k'], color = ['blue', 'green', 'red'])
plt.xlabel('Ethnicity')
plt.ylabel('Homicide Rate per 100,000 People')
plt.title('Homicide Rate per 100,000 People by Ethnicity')
plt.ylim(0, max(df_to_visualize['homicide_rate_per_100k']) * 1.2)
plt.show()

In [ ]:
state_3 = police_shootings[police_shootings['state'] == "FL"]

#popestimate 2022
pop_black = 3241015
pop_hispanic = 6025039
pop_white = 11309403


df_to_visualize['homicides'] = [h_counts, w_counts, b_counts]
df_to_visualize['population'] = [pop_hispanic, pop_white, pop_black]

df_to_visualize['homicide_rate_per_100k'] = df_to_visualize['homicides'] / df_to_visualize['population'] * 100000

Questo risultato mostra che in Florida su **100,000 persone** vengono uccise:
- 3 persone ispaniche
- 2 persone bianche
- 5 persone afro-americane

In [ ]:
plt.figure(figsize = (5, 5))
plt.bar(df_to_visualize['ethnicity'], df_to_visualize['homicide_rate_per_100k'], color = ['blue', 'green', 'red'])
plt.xlabel('Ethnicity')
plt.ylabel('Homicide Rate per 100,000 People')
plt.title('Homicide Rate per 100,000 People by Ethnicity')
plt.ylim(0, max(df_to_visualize['homicide_rate_per_100k']) * 1.2) 
plt.show()

Questi risultati evidenziano una discriminazione nei confronti delle minoranze etniche, che risultano essere maggiormente incentrate sulle persone afro-americane e ispaniche.

## 2.6 Implementazione di un regressore lineare 

Il passo finale è l'implementazione di un regressore lineare. Il regressore lineare permette di modellare e comprendere se esistono relazioni tra le variabili presenti all'interno del dataset. L'obiettivo è, dunque, quello di stabilire una relazione di tipo lineare tra una variabile dipendente, da predire, e una o più variabili indipendenti utilizzate per la predizione. Nel caso specifico la variabile dipendente **y** verrà rappresentata dal numero di morti totali di una determinata etnia, la popolazione afroamericana. Le variabili indipendenti utilizzate per predire la variabile *y* saranno: 
 $$x_1 : \text{latitudine  } \text{(intercetta)} $$ 
 $$x_2 : \text{rapporto popolazione bianca e popolazione afroamericana} $$ 

Si deve dunque mostrare una relazione, se esiste, tra l'aumento e la diminuzione della latitudine, e se presente allora indicherà una maggiore pericolosità a nord/sud, ed infine tra la differenza di numerosità della prima popolazione, quasi sempre bianca, e la minoranza individuata dagli afroamericani; se questa permette di diminuire il numero di morti con una maggiore integrazione della popolazione afroamericana negli stati, allora potrebbe provare una possibile discriminazione nei confronti della minoranza presa in studio. <br>
Si prendono in considerazione tutti e 50 gli Stati, la loro posizione con `latitude`, e l'`ethnicity_ratio` che viene invece calcolata utilizzando i dati di censimento delle popolazioni raccolti da Census Reporter[4]. Costruito questo dataset, si allena il modello di regressione.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [ ]:
data = {
    
    'latitude': [36.778259, 47.5515, 31.9686, 45.2538, 27.6648, 34.0489, 43.9695,
                 43.0760, 32.1574, 35.0078, 47.7511, 39.5501, 44.5588, 43.1939, 41.5801, 41.599998,
                35.782169, 44.068203, 44.000000, 46.392410, 44.182205, 40.730610, 40.367474,
                41.203323, 40.000000, 30.391830, 42.407211, 38.573936, 35.860119, 37.926868, 44.500000,
                40.273502, 66.160507, 32.318230, 34.799999, 39.000000, 42.032974, 38.500000, 37.839333,
                33.000000, 46.965260, 39.876019, 41.500000, 39.419220, 39.000000, 39.833851, 34.307144, 19.741755,
                44.500000, 39.045753],

    'ethnicity_ratio': [13160426/2025218, 639297/25426, 11683430/3508706, 1249233/21583,
                       11309403/3241015, 3814587/322459, 726885/18053, 473220/3990,
                       5415657/3349240, 2517403/268543, 4941456/299537, 3793270/220756,
                       583546/6264, 1208101/17931, 745825/51326, 2246913/353973, 6497519/2155650,
                       1531219/11868, 3037571/78228, 4356347/395179, 7289051/1310464, 10408933/2633840,
                       8948793/1401403, 9487228/1311581, 7356301/1660423, 2603221/1420055, 4676627/459740,
                       4733411/656739, 5070026/1089738, 5095130/1594785, 4654031/345678, 5192455/628054, 421104/20664,
                       3250182/1296500, 2054922/435637, 599347/220010, 2650110/119004, 2147943/147666, 3710998/343194,
                       1625979/1072962, 937365/3428, 1410145/286471, 1492005/89520, 2557466/34873, 1594128/59390, 4766476/1111632,
                       735478/36870, 297937/23682, 726885/18053, 2906492/1800914],
    
    'deaths': [police_shootings[police_shootings['state'] == "CA"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "ND"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "TX"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "ME"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "FL"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "AZ"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "SD"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "WY"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "GA"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "OK"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "WA"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "CO"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "VT"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "NH"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "RI"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "CT"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "NC"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "ID"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "OR"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "MN"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "MI"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "NY"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "OH"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "PA"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "IL"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "LA"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "MA"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "MO"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "TN"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "VA"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "WI"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "IN"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "AK"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "AL"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "AR"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "DE"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "IA"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "KS"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "KY"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "MS"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "MT"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "NV"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "NE"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "UT"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "WV"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "NJ"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "NM"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "HI"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "SC"]['race'].value_counts().get('B', 0),
            police_shootings[police_shootings['state'] == "MD"]['race'].value_counts().get('B', 0)]
}


df = pd.DataFrame(data)

In [ ]:
X = df[['latitude', 'ethnicity_ratio']]
y = df['deaths']


# Dividere i dati in set di addestramento e di test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

# Creare il modello di regressione lineare
model = LinearRegression()

# Addestrare il modello
model.fit(X_train, y_train)

# Predire sui dati di test
y_pred = model.predict(X_test)

# Valutare il modello
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


# Visualizzare i coefficienti del modello
print('Coefficients:', model.coef_)
print('Intercept:', model.intercept_)

In [ ]:
import statsmodels.api as sm

# Aggiungere una costante per l'intercetta
X = sm.add_constant(df[['latitude', 'ethnicity_ratio']])
y = df['deaths']

# Creare il modello OLS
model = sm.OLS(y, X).fit()

# Stampare il sommario del modello
print(model.summary())

Interpretiamo i risultati nella seguente maniera: una diminuzione di circa **2** punti sulla latitude diminuisce il numero di morti suggerendo una sicurezza maggiore negli stati del Nord, con livello statistico significativo (**p-value pari a 0,011**). Una diminuzione della disparità tra popolazione bianca e popolazione afroamericana di circa **0,23** punti mostra un livello di morti inferiore suggerendo un lieve miglioramento se si incrementa il numero di etnie afroamericane, con livello statistico significativo (**p-value pari a 0,045**). <br>
La statistica dell'R² si ferma comunque alla rappresentazione del 26 % della varianza della variabile *Y*.
 

****

# 3. Conclusioni e considerazioni future

Dallo studio condotto si evince, dunque, un uso spropositato delle armi sia da parte degli ufficiali di polizia che da parte degli individui coinvolti. L'uso comune delle armi è sicuramente una caratteristica legata agli Stati Uniti, in quanto vige la libera circolazione e detenzione delle armi, la cui regolamentazione non è affatto stretta e difficile da raggirare. Questo fattore, di stampo prettamente socio-culturale, potrebbe in qualche modo legarsi agli istinti violenti protagonisti di diversi episodi come nel caso recente di *Dexter Reed*, un 26enne afro-americano colpito da ben 96 colpi di pistola in 42 secondi[5]. Certamente le situazioni di pericolo e minaccia costante non rendono il lavoro ricoperto dalle forze dell'ordine qualcosa di semplice. Rimanere assolutamente lucidi in situazioni pericolose e agire in breve tempo per garantire la sicurezza di tutti è complesso. Tuttavia, l'esistenza di un problema di violenza e pregiudizio ai danni delle minoranze etniche è reale e spesso sottovalutato. <br>
Lo studio può essere ulteriormente migliorato incrementando la dimensione del dataset così da fornire una rappresentazione ancora più accurata (*aumentare la statistica dell'R²*) da parte del regressore lineare implementato nell'ultima fase, permettendo così di rendere statisticamente evidente una discriminazione ai danni delle minoranze. 

# Bibliografia

[1] Fatal Force Database, The Washington Post, [ https://github.com/washingtonpost/data-police-shootings ] <br>
[2] “Mapping Police Violence,” Campaign Zero, [Date accessed], [ https://mappingpoliceviolence.org/ ] <br>
[3] Choropleth Source, [ https://www.kaggle.com/code/yashgpt/choropleth-maps-geographic-visualization ] <br>
[4] Census Reporter, [ https://censusreporter.org/ ] <br>
[5] La polizia Usa uccide un afroamericano con 96 colpi, [ https://www.ansa.it/sito/notizie/mondo/nordamerica/2024/04/10/video-shock-polizia-usa-uccide-afroamericano-con-96-colpi_8b997b89-0acb-4518-ad5d-5f44139795a8.html ] <br>
<br>
*Lo studio nasce sull'ispirazione del dataset: Police Violence & Racial Equity - Part 1 of 2* [ https://www.kaggle.com/datasets/jpmiller/police-violence-in-the-us/data?select=police_killings_MPV.csv ]